### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas` for data manipulation, `cosine_similarity` from `sklearn.metrics.pairwise` for similarity computation, and `numpy` for numerical operations.

2. **Load Data**:
   - Load user-event interaction data from the Excel file `Combined_Interactions.xlsx`.

3. **Create User-Item Matrix**:
   - Pivot the data to create a matrix (`user_item_matrix`) where rows are users, columns are event IDs, and values are interaction scores. Missing values are filled with 0.

4. **Compute User Similarity**:
   - Compute a **user-user similarity matrix** using cosine similarity based on event interaction patterns.

5. **Recommend Events Function**:
   - The function `recommend_events(user_index, top_n=10)` takes a user's index and returns the top `n` recommended events.
   
   - **Flow**:
     - Identify users similar to the target user by sorting their cosine similarity scores.
     - Find events that similar users have interacted with but the target user has not.
     - Accumulate recommendation scores for these events based on similar users' interaction scores.

6. **Recommendation Filtering**:
   - Exclude events that the user has already interacted with or events with a score of `-1`.
   
7. **Sorting and Output**:
   - Sort recommended events by their accumulated interaction scores and return the top `n` recommendations.

8. **Example Usage**:
   - Recommends events for a specific user (e.g., `'Ketaki'`), and prints the top recommendations.

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("transformed_interactions_sc (1).xlsx")

In [6]:
df.drop(columns=['Secondary Category'])

,Name,Event ID,Interaction,Interaction Score,Primary Category,Sub Category
0,Aakash,EVT0014,Create Circle,7,Travel,Camping
1,Aakash,EVT0015,Join Circle,6,Adventure,Hiking and Trekking
2,Aakash,EVT0018,Booked,5,Tech and Innovation,Exhibition
3,Aakash,EVT0020,Disliked,-1,Fitness,Marathon
4,Aakash,EVT0029,Liked,3,Tech and Innovation,Exhibition
...,...,...,...,...,...,...
185,Vrushali,EVT0023,Skip,0,Fitness,Zumba
186,Vrushali,EVT0036,Opened,1,Dance,Classical
187,Vrushali,EVT0147,Invite Friends,7,Music and Entertainment,Shopping
188,Vrushali,EVT0124,Disliked,-1,Music and Entertainment,Magic and Illusion


In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_excel('transformed_interactions_sc (1).xlsx')

# Filter out events with interaction score of -1
# data = data[data['Interaction Score'] != -1]

# Pivot table to create user-item interaction matrix
user_item_matrix = data.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

# Compute user-user similarity matrix using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)

# Function to recommend events for a given user
def recommend_events(user_index, top_n=10):
    # Find similar users
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}
    
    # Iterate over similar users and recommend events
    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if user_item_matrix.iloc[user_index][event_id] == 0 and user_item_matrix.iloc[similar_user][event_id] > 0:
                if event_id not in event_recommendations and user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] = user_item_matrix.iloc[similar_user][event_id]
                elif user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] += user_item_matrix.iloc[similar_user][event_id]
    
    # Sort recommendations by interaction score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Function to find pairs of users with most similarity
def find_similar_users_pairs():
    # Compute all pairwise similarities
    pairwise_similarity = cosine_similarity(user_item_matrix)
    
    # Get indices of users
    users = user_item_matrix.index
    
    # Initialize list to store pairs and their similarity scores
    similar_pairs = []
    
    # Iterate over all pairs of users
    for i in range(len(users)):
        for j in range(i + 1, len(users)):
            similar_pairs.append(((users[i], users[j]), pairwise_similarity[i, j]))
    
    # Sort pairs by similarity score in descending order
    similar_pairs = sorted(similar_pairs, key=lambda x: x[1], reverse=True)
    
    return similar_pairs

# Example: Recommend events for user 'Pratik Bhande'
user = 'Pratik Bhande'
user_index = user_item_matrix.index.get_loc(user)  # Get the index of the user in the user_item_matrix
recommended_events = recommend_events(user_index)
print("Recommended events for {}: {}".format(user, recommended_events))

# Example: Find pairs of users with most similarity
similar_user_pairs = find_similar_users_pairs()
print("\nPairs of users with most similarity:")
for pair, similarity_score in similar_user_pairs:
    print(pair, similarity_score)


Recommended events for Pratik Bhande: [('EVT0015', 55), ('EVT0022', 47), ('EVT0147', 34), ('EVT0142', 24), ('EVT0010', 18), ('EVT0033', 17), ('EVT0037', 14), ('EVT0129', 14), ('EVT0100', 14), ('EVT0030', 13)]

Pairs of users with most similarity:
('Dinesh', 'Krishna') 0.8412505710193438
('Pratik Bhande', 'Rupesh') 0.6913536713557387
('Krishna', 'Priyanka') 0.6870788926077166
('Junaid', 'Tanushree') 0.6184342918775317
('Rupesh', 'Santosh') 0.616444713634013
('Chinmay', 'Dinesh') 0.6058143439072292
('Tanushree', 'Vrushali') 0.6037075533048724
('Dinesh', 'Priyanka') 0.6003570366278481
('Junaid', 'Vasudev') 0.5675982346618825
('Aakash', 'Pratik Saboo') 0.551778093374035
('Krishna', 'Tanushree') 0.5451224390390118
('Avinash', 'Shreyas') 0.5436197090427292
('Ankita', 'Krishna') 0.5359330530755151
('Ankita', 'Dinesh') 0.5297434373852462
('Dinesh', 'Tanushree') 0.5228260776185676
('Chinmay', 'Priyanka') 0.5222781808854776
('Avinash', 'Rupesh') 0.4798409384402382
('Dinesh', 'Sujan') 0.479212696

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_excel('transformed_interactions_sc (1).xlsx')

# Filter out events with interaction score of -1
#data = data[data['Interaction Score'] != -1]

# Pivot table to create user-item interaction matrix
user_item_matrix = data.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

# Compute user-user similarity matrix using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)

# Function to recommend events for a given user
def recommend_events(user_index, top_n=10):
    # Find similar users
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}
    
    # Iterate over similar users and recommend events
    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if user_item_matrix.iloc[user_index][event_id] == 0 and user_item_matrix.iloc[similar_user][event_id] > 0:
                if event_id not in event_recommendations and user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] = user_item_matrix.iloc[similar_user][event_id]
                elif user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] += user_item_matrix.iloc[similar_user][event_id]
    
    # Sort recommendations by interaction score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Example: Recommend events for user 'Aakash'
user = 'Ketaki'
user_index = user_item_matrix.index.get_loc(user) # Get the index of the user in the user_item_matrix
recommended_events = recommend_events(user_index)
print("Recommended events for {}: {}".format(user, recommended_events))


Recommended events for Ketaki: [('EVT0015', 55), ('EVT0014', 50), ('EVT0022', 47), ('EVT0018', 46), ('EVT0020', 38), ('EVT0147', 34), ('EVT0142', 24), ('EVT0007', 21), ('EVT0033', 17), ('EVT0023', 16)]


In [21]:
user = 'Vrushali'
user_index = user_item_matrix.index.get_loc(user) # Get the index of the user in the user_item_matrix
recommended_events = recommend_events(user_index)
print("Recommended events for {}: {}".format(user, recommended_events))

Recommended events for Vrushali: [('EVT0018', 46), ('EVT0020', 38), ('EVT0007', 21), ('EVT0010', 18), ('EVT0033', 17), ('EVT0023', 16), ('EVT0037', 14), ('EVT0100', 14), ('EVT0129', 14), ('EVT0029', 13)]


In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load data
data = pd.read_excel('Combined_Interactions.xlsx')

# Filter out events with interaction score of -1
#data = data[data['Interaction Score'] != -1]

# Pivot table to create user-item interaction matrix
user_item_matrix = data.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

# Compute user-user similarity matrix using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)

# Function to recommend events for a given user
# Function to recommend events for a given user
def recommend_events(user_index, top_n=10):
    # Find similar users
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself
    
    # Initialize dictionary to store event recommendations
    event_recommendations = {}
    
    # Get the events already interacted with by the user
    #interacted_events = user_item_matrix.columns[user_item_matrix.iloc[user_index].to_numpy().nonzero()]
    interacted_events = data[data['Name'] == user]['Event ID'].tolist()
    # Iterate over similar users and recommend events
    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if (user_item_matrix.iloc[user_index][event_id] == 0 and 
                user_item_matrix.iloc[similar_user][event_id] > 0 and 
                event_id not in interacted_events):  # Check if event is not already interacted with
                if event_id not in event_recommendations and user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] = user_item_matrix.iloc[similar_user][event_id]
                elif user_item_matrix.iloc[similar_user][event_id] != -1:
                    event_recommendations[event_id] += user_item_matrix.iloc[similar_user][event_id]
    
    # Sort recommendations by interaction score
    sorted_recommendations = sorted(event_recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Return top n recommendations
    return sorted_recommendations[:top_n]

# Example: Recommend events for user 'Vasudev'
user = 'Ketaki'
user_index = user_item_matrix.index.get_loc(user) # Get the index of the user in the user_item_matrix
recommended_events = recommend_events(user_index)
print("Recommended events for {}: {}".format(user, recommended_events))


Recommended events for Ketaki: [('EVT0022', 47), ('EVT0085', 45), ('EVT0030', 44), ('EVT0018', 41), ('EVT0020', 38), ('EVT0147', 34), ('EVT0025', 28), ('EVT0100', 25), ('EVT0142', 24), ('EVT0024', 20)]
